In [3]:
import pandas as pd
import openpyxl
import awswrangler as awr

pegando excel

In [4]:
excel_path = r"C:\Users\raphael.almeida\Documents\Processos\relatorio_inadimplencia\historico_faturas.xlsx"

df_base = pd.read_excel(excel_path, engine='openpyxl')

gerando base de inadimplência de hoje

In [5]:
query_path = r"C:\Users\raphael.almeida\Documents\Processos\relatorio_inadimplencia\sql\faturas_inadimplentes.sql"

with open(query_path, 'r') as file:
    query = file.read()

df_inadimplentes = awr.athena.read_sql_query(query, database='silver')
df_inadimplentes = df_inadimplentes.drop_duplicates('ponteiro', keep='first')

In [6]:
today = pd.to_datetime('today').date()
month = today.month
df_inadimplentes['data_vencimento'] = pd.to_datetime(df_inadimplentes['data_vencimento'])


#df_inadimplentes = df_inadimplentes[df_inadimplentes['data_vencimento']>= pd.to_datetime('2025-01-31')]
df_inadimplentes = df_inadimplentes[df_inadimplentes['data_vencimento'].dt.month == month]

df_inadimplentes['situacao'] = pd.NA
df_inadimplentes['data_atualizacao'] = pd.NA


concatenando e removendo linhas inteiras duplicadas

In [8]:
df_inadimplentes_composto =  pd.concat([df_base, df_inadimplentes])
df_inadimplentes_composto = df_inadimplentes_composto.sort_values(by='data_vencimento', ascending=False)
df_inadimplentes_composto = df_inadimplentes_composto.drop_duplicates(keep='first')


C:\Users\raphael.almeida\AppData\Local\Temp\ipykernel_17524\512907624.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_inadimplentes_composto =  pd.concat([df_base, df_inadimplentes])


filtrando apenas os ponteiros de inadimplentes do mês e gerando uma lista

In [9]:
#quero filtrar apenas a coluna de inadimplentes cuja coluna de data_vencimento da mesma linha não seja null

df_inadimplentes_lista = df_inadimplentes_composto.loc[df_inadimplentes_composto['data_vencimento'].notna() ,'ponteiro'].to_list()



[187917, 252814, 61362, 349112, 365249, 203901, 354190, 92653, 301086, 355145, 391824, 251329, 352345, 324824, 302643, 252239, 322671, 350471, 257796, 253619, 392109, 229143, 415083, 352781, 231218, 291383, 216963, 304145, 226799, 166572, 323216, 172156, 302467, 351304, 392325, 228168, 416310, 255895, 253123, 322683, 300974, 389371, 226031, 389396, 168605, 251083, 251131, 350584, 351554, 382458, 90279, 125937, 406588, 61219, 126133, 323138, 121672, 60984, 302312, 129488, 255394, 246216, 224476, 351106, 392571, 406492, 302264, 254188, 272170, 275715, 227178, 323951, 412467, 346754, 349786, 254530, 224439, 225836, 303249, 349900, 269739, 253719, 322501, 391803, 391600, 410959, 61271, 253136, 414528, 350192, 255921, 94160, 252293, 382290, 353558, 125113, 303436, 300428, 355049, 350304, 226762, 300901, 221165, 414516, 219583, 378715, 346624, 302581, 237997, 349303, 324061, 303049, 324298, 228950, 92542, 275839, 275409, 254133, 350608, 252877, 251483, 392036, 379652, 302529, 166390, 199691,

gerando o relatório de pagamentos do mês

In [ ]:
query_path = r"C:\Users\raphael.almeida\Documents\Processos\relatorio_inadimplencia\sql\faturas_baixadas.sql"

with open(query_path, 'r') as file:
    query = file.read()

df_pagamentos = awr.athena.read_sql_query(query, database='silver')


In [ ]:
df_pagamentos['data_baixa'] = pd.to_datetime(df_pagamentos['data_baixa'])
df_pagamentos = df_pagamentos.drop_duplicates('ponteiro', keep='first')
#df_pagamentos = df_pagamentos[df_pagamentos['data_baixa']>= pd.to_datetime('2025-01-31').date()]
df_pagamentos = df_pagamentos[df_pagamentos['data_baixa'].dt.month == month]
df_pagamentos_mensal = df_pagamentos[df_pagamentos['ponteiro'].isin(df_inadimplentes_lista)]


In [ ]:
df_pagamentos.loc[:, 'data_atualizacao'] = df_pagamentos['data_baixa']
df_pagamentos.loc[:, 'situacao']='PAGO'

df_inadimplentes.loc[:, 'data_atualizacao'] = df_inadimplentes['data_vencimento']
df_inadimplentes.loc[:, 'situacao']='INADIMPLENTE'

concatenando faturas baixadas dos vencidos deste mes com a base composta e atualizada

In [ ]:
df_atualizado = pd.concat([df_inadimplentes_composto, df_pagamentos_mensal])

retornando ao excel base atualizado

In [ ]:
excel_save_path = r"C:\Users\raphael.almeida\Documents\Processos\relatorio_inadimplencia\historico_faturas.xlsx"
df_atualizado.to_excel(excel_save_path, engine='openpyxl', index=False, sheet_name='historico_faturas')